<a href="https://colab.research.google.com/github/axccun/PyTorch/blob/main/QA_system_with_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
import pandas as pd

In [99]:
df = pd.read_csv('/content/QA.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [100]:
def tokenization(text):
  text = text.lower()
  text = text.replace("'",'')
  text = text.replace("?",'')
  text = text.replace(".",'')
  return text.split()

In [101]:
tokenization("Hello MY name is AKash RIght?")

['hello', 'my', 'name', 'is', 'akash', 'right']

In [102]:
vocab = {'<UNK>':0}

In [103]:
def build_vocab(row):
  token_question = tokenization(row['question'])
  token_answer = tokenization(row['answer'])
  all_tokens = token_question + token_answer
  for token in all_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)
df.apply(build_vocab,axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
101,None
102,None
103,None
104,None


In [104]:
len(vocab.keys())

342

In [105]:
def text_to_number(text,vocab):
  indices = []
  for token in tokenization(text):
    if token in vocab:
      indices.append(vocab[token])
    else:
      indices.append(vocab['<UNK>'])
  return indices

In [97]:
# Creating a pytorch dataset
import torch
from torch.utils.data import Dataset,DataLoader
class QA_Dataset(Dataset):
  def __init__(self,df,vocab):
    self.df = df
    self.vocab = vocab
  def __len__(self):
    return self.df.shape[0]
  def __getitem__(self,index):
    numerical_question = text_to_number(self.df.iloc[index]['question'],self.vocab)
    answer_text = str(self.df.iloc[index]['answer']).replace(" ", "").replace(".", "")
    answer_token = tokenization(answer_text)
    answer_index = self.vocab.get(answer_token[0], self.vocab['<UNK>'])
    return torch.tensor(numerical_question),torch.tensor(answer_index)

In [106]:
dataset = QA_Dataset(df,vocab)

In [107]:
dataset[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor(7))

In [108]:
dataloader = DataLoader(dataset,batch_size=1,shuffle=True)

In [109]:
len(dataloader)

106

In [110]:
import torch.nn as nn

In [111]:
class MyFirstRNN(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size,embedding_dim=50)
    self.rnn = nn.RNN(50,64,batch_first=True)
    self.fc = nn.Linear(64,vocab_size)
  def forward(self,question):
    embedded_question = self.embedding(question)
    hidden,final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [119]:
learning_rate = 0.001
epochs = 100

In [120]:
model = MyFirstRNN(len(vocab))

In [121]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [122]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer)
    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 587.809469
Epoch: 2, Loss: 379.593239
Epoch: 3, Loss: 315.938753
Epoch: 4, Loss: 271.044695
Epoch: 5, Loss: 231.202499
Epoch: 6, Loss: 193.576547
Epoch: 7, Loss: 161.341391
Epoch: 8, Loss: 130.884408
Epoch: 9, Loss: 104.865877
Epoch: 10, Loss: 83.128116
Epoch: 11, Loss: 67.554186
Epoch: 12, Loss: 53.503144
Epoch: 13, Loss: 43.765787
Epoch: 14, Loss: 35.764893
Epoch: 15, Loss: 29.678660
Epoch: 16, Loss: 24.981723
Epoch: 17, Loss: 21.213954
Epoch: 18, Loss: 17.715082
Epoch: 19, Loss: 15.161847
Epoch: 20, Loss: 13.010135
Epoch: 21, Loss: 11.308666
Epoch: 22, Loss: 9.876724
Epoch: 23, Loss: 8.589701
Epoch: 24, Loss: 7.584410
Epoch: 25, Loss: 6.721542
Epoch: 26, Loss: 5.924651
Epoch: 27, Loss: 5.307203
Epoch: 28, Loss: 4.667297
Epoch: 29, Loss: 4.189148
Epoch: 30, Loss: 3.777843
Epoch: 31, Loss: 3.411272
Epoch: 32, Loss: 3.100864
Epoch: 33, Loss: 2.826121
Epoch: 34, Loss: 2.585088
Epoch: 35, Loss: 2.368657
Epoch: 36, Loss: 2.185716
Epoch: 37, Loss: 2.007651
Epoch: 38, Loss: 

In [123]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_number(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [124]:
predict(model, "what is the symbol of oxygen?")

I don't know
o
